In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import hashlib
import gc
samplerows=np.nan

In [2]:
store_info=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/OtherInput/store_list_from_SFTP/MediaStormStores20181101-134628-331.txt",sep="|",dtype=str)
store_info=store_info[['location_id','city_nm','state_nm','zip_cd','open_dt']]

store_info_2=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/OtherInput/store_list_from_SFTP/MediaStormStores20181001-135417-132.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/OtherInput/store_list_from_SFTP/MediaStormStores20180901-133640-935.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/OtherInput/store_list_from_SFTP/MediaStormStores20180801-133641-576.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/OtherInput/store_list_from_SFTP/MediaStormStores20180703.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/OtherInput/store_list_from_SFTP/MediaStormStores20171115.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/OtherInput/store_list_from_SFTP/MediaStormStores20170913.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)
store_info['zip_cd']=store_info['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))


DMA_Zip=pd.read_excel("/Users/Jian/Desktop/Media_Storm/Docs/Geo mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
DMA_Zip=DMA_Zip.iloc[:,[0,2]]
DMA_Zip.columns=['zip_cd','DMA']
DMA_Zip=DMA_Zip.drop_duplicates(['zip_cd'])


store_info=pd.merge(store_info,DMA_Zip,on="zip_cd",how="left")

In [3]:
stores_sales_data=pd.read_excel("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/Datorama/2018/outputs_2018-11-10/wide_sales_date2018-11-10.xlsx",dtype=str,sheetname="sales")
stores_trans_data=pd.read_excel("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/Datorama/2018/outputs_2018-11-10/wide_sales_date2018-11-10.xlsx",dtype=str,sheetname="transactions")

Q3_Start_week_2018=datetime.date(2018,8,11)
Q3_End_week_2018=datetime.date(2018,11,3)
Q3_Start_week_2017=datetime.date(2017,8,12)
Q3_End_week_2017=datetime.date(2017,11,4)
Q3_2017_Weeks=[str(Q3_Start_week_2017+datetime.timedelta(days=7*i)) for i in range(13)]
Q3_2018_Weeks=[str(Q3_Start_week_2018+datetime.timedelta(days=7*i)) for i in range(13)]


stores_sales_data=stores_sales_data[['location_id']+Q3_2017_Weeks+Q3_2018_Weeks]
stores_trans_data=stores_trans_data[['location_id']+Q3_2017_Weeks+Q3_2018_Weeks]


for col in Q3_2017_Weeks+Q3_2018_Weeks:
    stores_sales_data[col]=stores_sales_data[col].astype(float)
    stores_trans_data[col]=stores_trans_data[col].astype(float)

stores_trans_data['2017_Q3_Trans']=stores_trans_data[Q3_2017_Weeks].sum(axis=1)
stores_trans_data['2018_Q3_Trans']=stores_trans_data[Q3_2018_Weeks].sum(axis=1)
stores_sales_data['2017_Q3_Sales']=stores_sales_data[Q3_2017_Weeks].sum(axis=1)
stores_sales_data['2018_Q3_Sales']=stores_sales_data[Q3_2018_Weeks].sum(axis=1)

Q3_Sales_Trans=pd.merge(stores_trans_data[['location_id','2017_Q3_Trans','2018_Q3_Trans']],
                       stores_sales_data[['location_id','2017_Q3_Sales','2018_Q3_Sales']],
                       on="location_id",how="outer")

store_list_set=set(Q3_Sales_Trans['location_id'].unique().tolist())

df_1=pd.merge(store_info,Q3_Sales_Trans,on="location_id",how="left")
del store_info
df_1['Q3_Sales_YoY']=(df_1['2018_Q3_Sales']-df_1['2017_Q3_Sales'])/df_1['2017_Q3_Sales']
df_1['Q3_Trans_YoY']=(df_1['2018_Q3_Trans']-df_1['2017_Q3_Trans'])/df_1['2017_Q3_Trans']
del store_info_2
gc.collect()

84

# V1 Exclude merch_cat 80040410

In [4]:
rewards_sales_from_SP=pd.read_csv("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                                 dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt','merch_cat'])
rewards_removed=rewards_sales_from_SP[rewards_sales_from_SP['merch_cat']=="80040410"]
rewards_sales_from_SP=rewards_sales_from_SP[rewards_sales_from_SP['merch_cat']!="80040410"]
del rewards_sales_from_SP['merch_cat']

rewards_sales_from_SP=rewards_sales_from_SP.drop_duplicates()

rewards_sales_from_SP['transaction_date']=rewards_sales_from_SP['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

rewards_sales_from_SP_2017Q3=rewards_sales_from_SP[(rewards_sales_from_SP['transaction_date']>=(Q3_Start_week_2017-datetime.timedelta(days=6))) & (rewards_sales_from_SP['transaction_date']<=Q3_End_week_2017)]
del rewards_sales_from_SP
gc.collect()


rewards_sales_from_SP_2017Q3['total_transaction_amt']=rewards_sales_from_SP_2017Q3['total_transaction_amt'].astype(float)
rewards_sales_from_SP_2017Q3=rewards_sales_from_SP_2017Q3.drop_duplicates()
gc.collect()
rewards_sales_from_SP_2017Q3.shape

(13727931, 5)

In [5]:
#Check
# When come back with server, check the time for the 2 transaction
check_dup=rewards_sales_from_SP_2017Q3[(rewards_sales_from_SP_2017Q3['customer_id_hashed']=="d75fa736c3c766c72c6fe34a55b00edc2e5f5d90e5fbbb98876e75bfd4b270f9") & (rewards_sales_from_SP_2017Q3['transaction_date']==datetime.date(2017,8,6))]


In [6]:
rewards_sales_from_SP_2018Q3=pd.read_table("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180814-131127-683.txt",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'],sep="|",dtype=str)
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.rename(columns={"transaction_dt":"transaction_date"})
rewards_sales_from_SP_2018Q3['transaction_date']=rewards_sales_from_SP_2018Q3['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

biweekly_rewards=pd.read_table("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180828-131144-584.txt",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'],sep="|",dtype=str)
biweekly_rewards=biweekly_rewards.rename(columns={"transaction_dt":"transaction_date"})
biweekly_rewards['transaction_date']=biweekly_rewards['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.append(biweekly_rewards)

biweekly_rewards=pd.read_table("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180911-133008-590.txt",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'],sep="|",dtype=str)
biweekly_rewards=biweekly_rewards.rename(columns={"transaction_dt":"transaction_date"})
biweekly_rewards['transaction_date']=biweekly_rewards['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.append(biweekly_rewards)

biweekly_rewards=pd.read_table("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180925-132958-543.txt",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'],sep="|",dtype=str)
biweekly_rewards=biweekly_rewards.rename(columns={"transaction_dt":"transaction_date"})
biweekly_rewards['transaction_date']=biweekly_rewards['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.append(biweekly_rewards)

biweekly_rewards=pd.read_table("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181009-132856-945.txt",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'],sep="|",dtype=str)
biweekly_rewards=biweekly_rewards.rename(columns={"transaction_dt":"transaction_date"})
biweekly_rewards['transaction_date']=biweekly_rewards['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.append(biweekly_rewards)

biweekly_rewards=pd.read_table("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181023-132316-829.txt",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'],sep="|",dtype=str)
biweekly_rewards=biweekly_rewards.rename(columns={"transaction_dt":"transaction_date"})
biweekly_rewards['transaction_date']=biweekly_rewards['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.append(biweekly_rewards)

biweekly_rewards=pd.read_table("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181106-132358-296.txt",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'],sep="|",dtype=str)
biweekly_rewards=biweekly_rewards.rename(columns={"transaction_dt":"transaction_date"})
biweekly_rewards['transaction_date']=biweekly_rewards['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.append(biweekly_rewards)


rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3[(rewards_sales_from_SP_2018Q3['transaction_date']>=(Q3_Start_week_2018-datetime.timedelta(days=6))) & (rewards_sales_from_SP_2018Q3['transaction_date']<=Q3_End_week_2018)]
rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3.drop_duplicates()

rewards_sales_from_SP_2018Q3['total_transaction_amt']=rewards_sales_from_SP_2018Q3['total_transaction_amt'].astype(float)
del rewards_sales_from_SP_2017Q3['transaction_id']
del rewards_sales_from_SP_2018Q3['transaction_id']

gc.collect()
# No 80040410 in 2018 data

7

In [7]:
def add_week_end_date(x):
    weekday_num=x.weekday()
    if weekday_num==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-weekday_num)
    return y

rewards_by_store_sales_2018Q3=rewards_sales_from_SP_2018Q3.groupby(['location_id'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_store_sales_2018Q3=rewards_by_store_sales_2018Q3.rename(columns={"total_transaction_amt":"Q3_rewards_sales_2018"})
rewards_by_store_trans_2018Q3=rewards_sales_from_SP_2018Q3.groupby(['location_id'])['total_transaction_amt'].count().to_frame().reset_index()
rewards_by_store_trans_2018Q3=rewards_by_store_trans_2018Q3.rename(columns={"total_transaction_amt":"Q3_rewards_trans_2018"})

rewards_by_store_sales_2017Q3=rewards_sales_from_SP_2017Q3.groupby(['location_id'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_store_sales_2017Q3=rewards_by_store_sales_2017Q3.rename(columns={"total_transaction_amt":"Q3_rewards_sales_2017"})
rewards_by_store_trans_2017Q3=rewards_sales_from_SP_2017Q3.groupby(['location_id'])['total_transaction_amt'].count().to_frame().reset_index()
rewards_by_store_trans_2017Q3=rewards_by_store_trans_2017Q3.rename(columns={"total_transaction_amt":"Q3_rewards_trans_2017"})


In [8]:
gc.collect()
ids_2017_q3=set(rewards_sales_from_SP_2017Q3['customer_id_hashed'].unique().tolist())
ids_2018_q3=set(rewards_sales_from_SP_2018Q3['customer_id_hashed'].unique().tolist())

all_stores_existing_Q3_id=ids_2018_q3.intersection(ids_2017_q3)
all_stores_new_Q3_id=ids_2018_q3.difference(ids_2017_q3)

rewards_sales_from_SP_2018Q3['id_label']=np.where(rewards_sales_from_SP_2018Q3['customer_id_hashed'].isin(all_stores_existing_Q3_id),"Q3_existing",
                                                 np.where(rewards_sales_from_SP_2018Q3['customer_id_hashed'].isin(all_stores_new_Q3_id),"Q3_2018new",np.nan))
rewards_sales_from_SP_2017Q3['id_label']=np.where(rewards_sales_from_SP_2017Q3['customer_id_hashed'].isin(all_stores_existing_Q3_id),"Q3_existing",np.nan)


In [9]:
existing_sales_by_store_2018Q3=rewards_sales_from_SP_2018Q3.groupby(['location_id','id_label'])['total_transaction_amt'].sum().to_frame().reset_index()
new_sales_by_store_2018Q3=existing_sales_by_store_2018Q3[existing_sales_by_store_2018Q3['id_label']=="Q3_2018new"]
existing_sales_by_store_2018Q3=existing_sales_by_store_2018Q3[existing_sales_by_store_2018Q3['id_label']=="Q3_existing"]


existing_sales_by_store_2017Q3=rewards_sales_from_SP_2017Q3.groupby(['location_id','id_label'])['total_transaction_amt'].sum().to_frame().reset_index()
existing_sales_by_store_2017Q3=existing_sales_by_store_2017Q3[existing_sales_by_store_2017Q3['id_label']=="Q3_existing"]

del new_sales_by_store_2018Q3['id_label']
del existing_sales_by_store_2018Q3['id_label']
del existing_sales_by_store_2017Q3['id_label']

new_sales_by_store_2018Q3=new_sales_by_store_2018Q3.rename(columns={"total_transaction_amt":"Q3_new_rewards_sales_2018"})
existing_sales_by_store_2018Q3=existing_sales_by_store_2018Q3.rename(columns={"total_transaction_amt":"Q3_existing_rewards_sales_2018"})
existing_sales_by_store_2017Q3=existing_sales_by_store_2017Q3.rename(columns={"total_transaction_amt":"Q3_existing_rewards_sales_2017"})



In [10]:
def count_unique(x):
    y=len(set(x))
    return y
NEW_rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3[rewards_sales_from_SP_2018Q3['id_label']=="Q3_2018new"]
EXISTING_rewards_sales_from_SP_2018Q3=rewards_sales_from_SP_2018Q3[rewards_sales_from_SP_2018Q3['id_label']=="Q3_existing"]
EXISTING_rewards_sales_from_SP_2017Q3=rewards_sales_from_SP_2017Q3[rewards_sales_from_SP_2017Q3['id_label']=="Q3_existing"]

new_id_count_2018=NEW_rewards_sales_from_SP_2018Q3.groupby(['location_id'])['customer_id_hashed'].apply(count_unique).to_frame().reset_index().rename(columns={"customer_id_hashed":"Q3_new_id_counts_2018"})
existing_id_count_2018=EXISTING_rewards_sales_from_SP_2018Q3.groupby(['location_id'])['customer_id_hashed'].apply(count_unique).to_frame().reset_index().rename(columns={"customer_id_hashed":"Q3_existing_id_counts_2018"})
existing_id_count_2017=EXISTING_rewards_sales_from_SP_2017Q3.groupby(['location_id'])['customer_id_hashed'].apply(count_unique).to_frame().reset_index().rename(columns={"customer_id_hashed":"Q3_existing_id_counts_2017"})

total_id_count_2018=rewards_sales_from_SP_2018Q3.groupby(['location_id'])['customer_id_hashed'].apply(count_unique).to_frame().reset_index().rename(columns={"customer_id_hashed":"2018_Q3_rewards_id_shopped"})
total_id_count_2017=rewards_sales_from_SP_2017Q3.groupby(['location_id'])['customer_id_hashed'].apply(count_unique).to_frame().reset_index().rename(columns={"customer_id_hashed":"2017_Q3_rewards_id_shopped"})


In [11]:
output=pd.merge(df_1,rewards_by_store_sales_2018Q3,on="location_id",how="left")
output=pd.merge(output,rewards_by_store_trans_2018Q3,on="location_id",how="left")
output=pd.merge(output,rewards_by_store_sales_2017Q3,on="location_id",how="left")
output=pd.merge(output,rewards_by_store_trans_2017Q3,on="location_id",how="left")

output=pd.merge(output,existing_sales_by_store_2018Q3,on="location_id",how="left")
output=pd.merge(output,existing_sales_by_store_2017Q3,on="location_id",how="left")
output=pd.merge(output,new_sales_by_store_2018Q3,on="location_id",how="left")

output=pd.merge(output,total_id_count_2018,on="location_id",how="left")
output=pd.merge(output,total_id_count_2017,on="location_id",how="left")
output=pd.merge(output,existing_id_count_2018,on="location_id",how="left")
output=pd.merge(output,existing_id_count_2017,on="location_id",how="left")
output=pd.merge(output,new_id_count_2018,on="location_id",how="left")


In [12]:
output['Shopped Rewards Growth']=output['2018_Q3_rewards_id_shopped']-output['2017_Q3_rewards_id_shopped']
output['Rewards Sales YoY']=(output['Q3_rewards_sales_2018']-output['Q3_rewards_sales_2017'])/output['Q3_rewards_sales_2017']
output['location_id']=output['location_id'].astype(int)
output=output.sort_values(['location_id'])

In [36]:
rewards_sales_from_SP_2018Q3['week_end_date']=rewards_sales_from_SP_2018Q3['transaction_date'].apply(lambda x: add_week_end_date(x))
rewards_sales_from_SP_2017Q3['week_end_date']=rewards_sales_from_SP_2017Q3['transaction_date'].apply(lambda x: add_week_end_date(x))

rewards_sales_2018Q3_by_store_by_week=rewards_sales_from_SP_2018Q3.groupby(['location_id','week_end_date'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_sales_2017Q3_by_store_by_week=rewards_sales_from_SP_2017Q3.groupby(['location_id','week_end_date'])['total_transaction_amt'].sum().to_frame().reset_index()


In [37]:
total_sales_long=pd.read_csv("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/Datorama/2018/outputs_2018-11-03/combined_sales_long_2018-11-03.csv",dtype=str)
total_sales_long=total_sales_long[['location_id','sales','week_end_date']].rename(columns={"sales":"total_weekly_sales"})
total_sales_long['week_end_date']=total_sales_long['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
total_sales_long['total_weekly_sales']=total_sales_long['total_weekly_sales'].astype(float)
rewards_sales_2018Q3_by_store_by_week=rewards_sales_2018Q3_by_store_by_week.rename(columns={"total_transaction_amt":"rewards_sales"})
rewards_sales_2017Q3_by_store_by_week=rewards_sales_2017Q3_by_store_by_week.rename(columns={"total_transaction_amt":"rewards_sales"})
rewards_sales_2018Q3_by_store_by_week=pd.merge(rewards_sales_2018Q3_by_store_by_week,total_sales_long,on=['location_id','week_end_date'],how="left")
rewards_sales_2017Q3_by_store_by_week=pd.merge(rewards_sales_2017Q3_by_store_by_week,total_sales_long,on=['location_id','week_end_date'],how="left")
sales_Q3_by_week=rewards_sales_2017Q3_by_store_by_week.append(rewards_sales_2018Q3_by_store_by_week)
sales_Q3_by_week['Exclusion']=np.where(sales_Q3_by_week['total_weekly_sales']<sales_Q3_by_week['rewards_sales'],"Exclusion","Inclusion")
sales_Q3_by_week['Rewards_sales_Pctg']=sales_Q3_by_week['rewards_sales']/sales_Q3_by_week['total_weekly_sales']


In [38]:
writer=pd.ExcelWriter("/Users/Jian/Desktop/Media_Storm/Big_Lots/Analysis/2018_Q3/Q3_Store_Quadrants/BL_data_for_2018Q3_Quadrants_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output.to_excel(writer,"data_for_Q3_quadrants",index=False)
rewards_removed.to_excel(writer,"merch_cat_80040410",index=False)
sales_Q3_by_week.to_excel(writer,"sales_Q3_by_week",index=False)
writer.save()

In [15]:
rewards_sales_from_SP=pd.read_csv("/Users/Jian/Desktop/Server_Project_Foder/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                                 dtype=str)
rewards_sales_from_SP['transaction_date']=rewards_sales_from_SP['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

rewards_sales_from_SP_2017Q3=rewards_sales_from_SP[(rewards_sales_from_SP['transaction_date']>=(Q3_Start_week_2017-datetime.timedelta(days=6))) & (rewards_sales_from_SP['transaction_date']<=Q3_End_week_2017)]
rewards_sales_from_SP_2017Q3['total_transaction_amt']=rewards_sales_from_SP_2017Q3['total_transaction_amt'].astype(float)
x=rewards_sales_from_SP_2017Q3.sort_values(['total_transaction_amt'],ascending=False).head(200)
y=x[x['transaction_date']!=datetime.date(2017, 8, 14)]

/Users/Jian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
# Check merch_cat

product_taxonomy=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/Data/static_files/MediaStorm Data Extract - Product Taxonomy.txt",sep="|",dtype=str)
subclass_name=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/Data/static_files/MediaStorm Data Extract - Subclass Names.txt",sep="|",dtype=str,encoding ='ISO-8859-1')
product_taxonomy=pd.merge(product_taxonomy,subclass_name,on=['class_code_id','subclass_id'],how="outer")
# product_taxonomy=product_taxonomy.rename(columns={"item_id":"merch_cat"})
product_taxonomy_x=product_taxonomy[product_taxonomy['item_id'].isin(x['merch_cat'])]

In [70]:
z=x[x['transaction_date']==datetime.date(2017, 8, 14)]
z['merch_cat'].unique()

array(['80040410'], dtype=object)

In [71]:
z.shape

(103, 8)

In [74]:
rewards_sales_from_SP.head(2)

,customer_id_hashed,transaction_date,transaction_time,transaction_id,location_id,total_transaction_amt,total_transaction_units,merch_cat
0,bc45d2747297addd8bcad579da77756b007d27cd634c71...,2018-03-20,21:02,2152,5158,22.25,13.0,12006001
1,f627e56e080cb699864d9c488345aee82c98fa66ebc2f2...,2018-03-10,18:23,6496,186,16.0,3.0,55003009


In [73]:
rewards_sales_from_SP.shape

(140301793, 8)

In [75]:
high_merch_cat=rewards_sales_from_SP[rewards_sales_from_SP['merch_cat']=="80040410"]
high_merch_cat.shape

(104, 8)

In [77]:
high_merch_cat['transaction_date'].unique()

array([datetime.date(2017, 8, 14), datetime.date(2017, 9, 7)], dtype=object)

In [78]:
high_merch_cat.to_csv("/Users/Jian/Desktop/Media_Storm/Big_Lots/Analysis/2018_Q3/high_value_rewards_transactions_JL_20181128.csv",index=False)

In [16]:
rewards_sales_from_SP_2017Q3.head(2)

,customer_id_hashed,transaction_date,transaction_time,transaction_id,location_id,total_transaction_amt,total_transaction_units,merch_cat
106727391,fdbfe6da6d44a5d66f2af90bae3a32bf221711c65bcfd3...,2017-08-06,12:21,2353,4436,11.78,3.0,15002002
106727454,2f8f042b6b7314ff1d37028c0e6c75fb8c6d888633ee2c...,2017-08-06,19:13,4368,462,18.60,32.0,17001006


In [17]:
rewards_sales_from_SP_2017Q3_no_80040410=rewards_sales_from_SP_2017Q3[rewards_sales_from_SP_2017Q3['merch_cat']!='80040410']

In [18]:
del rewards_sales_from_SP_2017Q3_no_80040410['merch_cat']
rewards_sales_from_SP_2017Q3_no_80040410=rewards_sales_from_SP_2017Q3_no_80040410.drop_duplicates()

In [26]:
rewards_sales_from_SP_2017Q3_no_80040410_by_store=rewards_sales_from_SP_2017Q3_no_80040410.groupby(['location_id','transaction_date'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_sales_from_SP_2017Q3_no_80040410_by_store['week_end_date']=rewards_sales_from_SP_2017Q3_no_80040410_by_store['transaction_date'].apply(lambda x: add_week_end_date(x))
rewards_sales_from_SP_2017Q3_no_80040410_by_store=rewards_sales_from_SP_2017Q3_no_80040410_by_store.groupby(['location_id','week_end_date'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_sales_from_SP_2017Q3_no_80040410_by_store=rewards_sales_from_SP_2017Q3_no_80040410_by_store.rename(columns={"total_transaction_amt":"rewards_sales"})


In [27]:
total_sales_long=pd.read_csv("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/Datorama/2018/outputs_2018-11-10/combined_sales_long_2018-11-10.csv",dtype=str)
total_sales_long['week_end_date']=total_sales_long['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
total_sales_long=total_sales_long[['location_id','week_end_date','sales']]
rewards_sales_from_SP_2017Q3_no_80040410_by_store=pd.merge(rewards_sales_from_SP_2017Q3_no_80040410_by_store,total_sales_long,on=['location_id','week_end_date'],how="left")

In [29]:
rewards_sales_from_SP_2017Q3_no_80040410_by_store.to_csv("/Users/Jian/Desktop/Media_Storm/Big_Lots/Analysis/2018_Q3/Q3_Store_Quadrants/Rewards_by_store_no_80040410_JL_20181128.csv",index=False)




In [37]:
high_merch_cat_382=rewards_sales_from_SP[rewards_sales_from_SP['location_id']=="382"]
high_merch_cat_382['total_transaction_amt']=high_merch_cat_382['total_transaction_amt'].astype(float)
high_merch_cat_382=high_merch_cat_382.sort_values(['total_transaction_amt'],ascending=False)
high_merch_cat_382_09302017=high_merch_cat_382[(high_merch_cat_382['transaction_date']>=datetime.date(2017,9,24)) & (high_merch_cat_382['transaction_date']<=datetime.date(2017,9,30))]

/Users/Jian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
del high_merch_cat_382_09302017['merch_cat']
high_merch_cat_382_09302017=high_merch_cat_382_09302017.drop_duplicates()
high_merch_cat_382_09302017['total_transaction_amt'].sum()

87100.26999999995

In [42]:
high_merch_cat_382_09302017.head(2)

,customer_id_hashed,transaction_date,transaction_time,transaction_id,location_id,total_transaction_amt,total_transaction_units
115779758,2421f603da36485b45bbba22a926e8581eb2f5fe132428...,2017-09-30,18:05,8093,382,2399.94,12.0
115482752,caf1ef4498864b5169997ba3a9ce82c733f0b992f8e694...,2017-09-30,13:17,8052,382,1809.94,10.0


In [44]:
del high_merch_cat_382_09302017['location_id']
high_merch_cat_382_09302017=high_merch_cat_382_09302017.drop_duplicates()
high_merch_cat_382_09302017['total_transaction_amt'].sum()

87100.26999999995

In [45]:
high_merch_cat_382_09302017.head(2)

,customer_id_hashed,transaction_date,transaction_id,total_transaction_amt,total_transaction_units
115779758,2421f603da36485b45bbba22a926e8581eb2f5fe132428...,2017-09-30,8093,2399.94,12.0
115482752,caf1ef4498864b5169997ba3a9ce82c733f0b992f8e694...,2017-09-30,8052,1809.94,10.0


In [47]:
count_df=high_merch_cat_382_09302017.groupby(['customer_id_hashed','transaction_date'])['transaction_id'].count().to_frame().reset_index()
count_df_more_than_1=count_df[count_df['transaction_id']>1]

In [48]:
count_df_more_than_1.shape

(111, 3)

In [50]:
count_df_more_than_1['customer_id_hashed'].tolist()[4]

'0aa2e9e5d0c5ff368cf5a85e4e6e3cd0479a64c45ae0e69fac24ae179d38178c'

In [51]:
high_merch_cat_382_09302017[high_merch_cat_382_09302017['customer_id_hashed']=='0aa2e9e5d0c5ff368cf5a85e4e6e3cd0479a64c45ae0e69fac24ae179d38178c']

,customer_id_hashed,transaction_date,transaction_id,total_transaction_amt,total_transaction_units
114396749,0aa2e9e5d0c5ff368cf5a85e4e6e3cd0479a64c45ae0e6...,2017-09-25,7952,479.99,1.0
114619042,0aa2e9e5d0c5ff368cf5a85e4e6e3cd0479a64c45ae0e6...,2017-09-25,7953,479.99,1.0
114695765,0aa2e9e5d0c5ff368cf5a85e4e6e3cd0479a64c45ae0e6...,2017-09-25,4169,479.99,1.0
115538689,0aa2e9e5d0c5ff368cf5a85e4e6e3cd0479a64c45ae0e6...,2017-09-30,8024,479.99,1.0


In [ ]:
⁨MediaStorm Weekly Extracts 2017-09-30⁩



In [59]:
weekly_0930_sales=pd.read_table("/Users/Jian/Desktop/Media_Storm/Big_Lots/Ongoing/Data/2017_Data/MediaStorm Weekly Extracts 2017-09-30/MediaStormSalesWeekly 2017-09-30.txt",dtype=str,sep="|")
weekly_0930_sales['class_gross_sales_amt']=weekly_0930_sales['class_gross_sales_amt'].astype(float)


In [60]:
weekly_0930_sales[weekly_0930_sales['location_id']=="382"]['class_gross_sales_amt'].sum()

67423.06

In [58]:
weekly_0930_sales.head(2)

,location_id,week_end_dt,fiscal_week_nbr,gross_sales_amt,gross_transaction_cnt,class_code_id,class_gross_sales_amt
0,1,2017-09-30,35,0000000000079292.72,2583,11001,0000000000001116.25
1,1,2017-09-30,35,0000000000079292.72,2583,11003,0000000000002717.75
